In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from matplotlib.ticker import MultipleLocator
import numpy as np
import seaborn as sns
sns.set_style("darkgrid")
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import stats
import math
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.isnull().sum()

label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64

In [46]:
X = df.drop(['label'], axis = 1)
y = df['label']

In [30]:
y

0        1
1        0
2        1
3        4
4        0
        ..
41995    0
41996    1
41997    7
41998    6
41999    9
Name: label, Length: 42000, dtype: int64

In [53]:
# Divide tus datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [52]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_dim=784),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='uniform'),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='uniform'),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='uniform'),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='uniform'),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='uniform'),
    tf.keras.layers.Dense(10, activation='softmax', kernel_initializer='uniform')
])

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [54]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 784)               0         
                                                                 
 dense_57 (Dense)            (None, 100)               78500     
                                                                 
 dense_58 (Dense)            (None, 100)               10100     
                                                                 
 dense_59 (Dense)            (None, 100)               10100     
                                                                 
 dense_60 (Dense)            (None, 100)               10100     
                                                                 
 dense_61 (Dense)            (None, 100)               10100     
                                                                 
 dense_62 (Dense)            (None, 10)              

In [56]:
model.fit(X_train, y_train, batch_size=25, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1344/1344 [==============================] - 4s 3ms/step - loss: 0.0601 - accuracy: 0.9847 - val_loss: 0.1481 - val_accuracy: 0.9696
Epoch 2/5
1344/1344 [==============================] - 4s 3ms/step - loss: 0.0639 - accuracy: 0.9834 - val_loss: 0.1416 - val_accuracy: 0.9675
Epoch 3/5
1344/1344 [==============================] - 4s 3ms/step - loss: 0.0546 - accuracy: 0.9849 - val_loss: 0.1569 - val_accuracy: 0.9681
Epoch 4/5
1344/1344 [==============================] - 4s 3ms/step - loss: 0.0493 - accuracy: 0.9868 - val_loss: 0.1684 - val_accuracy: 0.9712
Epoch 5/5
1344/1344 [==============================] - 4s 3ms/step - loss: 0.0450 - accuracy: 0.9879 - val_loss: 0.1519 - val_accuracy: 0.9733


In [57]:
y_pred = model.predict(X_test)

263/263 [==============================] - 1s 2ms/step


In [61]:
model.evaluate(X_test, y_test)

263/263 [==============================] - 1s 2ms/step - loss: 0.1519 - accuracy: 0.9733


[0.15192139148712158, 0.9733333587646484]

In [58]:
# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(conf_matrix)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [12]:
rfc_eval = cross_val_score(estimator = random, X = X_train, y = y_train, cv = 3)
rfc_eval.mean()

0.9605654761904763

In [59]:
report = classification_report(y_test, y_pred)
print(report)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

### Conjunto de prueba

In [16]:
sample = pd.read_csv('sample_submission.csv')
sample

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
27995,27996,0
27996,27997,0
27997,27998,0
27998,27999,0


In [14]:
X_Prueba = pd.read_csv('test.csv')
X_Prueba

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
image_id=[]
contador = 1
for i in range(28000):
    image_id.append(i+1)

In [24]:
image_id

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [15]:
prediction_prueba = random.predict(X_Prueba)

In [25]:
df_final_prediction = {'ImageId': image_id, 'Label': prediction_prueba}

In [26]:
data = pd.DataFrame(df_final_prediction)

In [27]:
data

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [28]:
#guardar archivo csv
data.to_csv("Mnist.csv", index=False)